# <center> Сегментация Клиентов Онлайн Магазина Подарков

## Постановка Задачи

Маркетинг — неотъемлемая часть любого бизнеса. Для повышения прибыли компании важно понимать своего клиента, его пожелания и предпочтения. С появлением электронной коммерции, или онлайн-продаж, стало намного проще собирать данные о клиентах, анализировать их, находить закономерности и реализовывать маркетинговые кампании.

Большинство интернет-магазинов используют инструменты веб-аналитики, чтобы отслеживать просмотры страниц, количество и поведение посетителей и коэффициент отказов. Но отчёта из Google Analytics или аналогичной системы может быть недостаточно для полного понимания того, как клиенты взаимодействуют с сайтом. Компаниям важно иметь возможность быстро и точно реагировать на перемены в поведении клиентов, создавая инструменты, которые обнаруживают эти изменения практически в режиме реального времени.

Машинное обучение помогает поисковой системе анализировать огромное количество данных о посетителях платформы, узнавать модели поведения профессиональных покупателей, определять категорию клиентов (например, лояльные/перспективные/новички/спящие/ушедшие) и выбирать правильную стратегию взаимодействия с ними.

Стоит также отметить, что компании, использующие машинное обучение на своих платформах электронной коммерции, могут постоянно повышать эффективность бизнес-процессов: настраивать товарную выборку персонально для каждого покупателя и предлагать выгодную цену в соответствии с бюджетом клиента и т. д. Эта задача относится к категории построения рекомендательных систем, речь о которых пойдёт в следующем разделе нашего курса.

> Как правило, наборы данных для электронной коммерции являются частной собственностью и, следовательно, их трудно найти среди общедоступных данных. Однако [The UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/index.php)  создал набор данных, содержащий фактические транзакции за 2010 и 2011 годы. С ним нам как раз и предлагается поработать в этом кейсе. 

> В нашем распоряжении будет набор данных, который содержит все транзакции, произошедшие в период с 01/12/2010 по 09/12/2011 для базирующейся в Великобритании компании, занимающейся онлайн-розничной торговлей. Компания в основном продает уникальные подарки на все случаи жизни. Многие клиенты компании являются оптовиками.


**Бизнес-задача:** произвести сегментацию существующих клиентов, проинтерпретировать эти сегменты и определить стратегию взаимодействия с ними.

**Техническая задача для вас как для специалиста в Data Science:** построить модель кластеризации клиентов на основе их покупательской способности, частоты заказов и срока давности последней покупки, определить профиль каждого из кластеров.

**Основные цели проекта:**
1. Произвести предобработку исходного набора данных о транзакциях.
2. Провести разведывательный анализ данных и выявить основные закономерности.
3. Сформировать набор данных о характеристиках каждого из уникальных клиентов.
4. Построить несколько моделей машинного обучения, решающих задачу кластеризации клиентов, определить количество кластеров и проинтерпретировать их.
5. Спроектировать процесс предсказания категории интересов клиента и протестировать вашу модель на новых клиентах.

## Описание Данных

Данные представляют собой таблицу в формате CSV, в каждой строке которой содержится информация об уникальной транзакции.

Признаки, описывающие каждую транзакцию:

* InvoiceNo — номер счёта-фактуры (уникальный номинальный шестизначный номер, присваиваемый каждой транзакции; буква "C" в начале кода указывает на отмену транзакции);
* StockCode — код товара (уникальное пятизначное целое число, присваиваемое каждому отдельному товару);
* Description — название товара;
* Quantity — количество каждого товара за транзакцию;
* InvoiceDate — дата и время выставления счёта/проведения транзакции;
* UnitPrice — цена за единицу товара в фунтах стерлингов;
* CustomerID — идентификатор клиента (уникальный пятизначный номер, однозначно присваиваемый каждому клиенту);
* Country — название страны, в которой проживает клиент.

In [1]:
#standard libraries
import pandas as pd, numpy as np
import datetime

#visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

#from mpl_toolkits.mplot3d import Axes3D
#plt.rcParams["patch.force_edgecolor"] = True

from IPython.display import display, HTML


#store results
#from collections import OrderedDict


#warnings
import warnings 
warnings.filterwarnings("ignore")

#data transformation
from sklearn import preprocessing

#clusterization
from sklearn import cluster
from sklearn import mixture
from sklearn import decomposition
from sklearn import manifold

#classification
from sklearn import linear_model
from sklearn import ensemble

#quality control
from sklearn import metrics

#model learning on train / test
from sklearn import model_selection

#user-made
from user_fx import get_quantity_cancelled
from user_fx import plot_cluster_profile
from user_fx import clusters_by_silhouette, clusters_by_calinski_harabasz, clusters_by_davies_bouldin
from user_fx import best_result_by_silhouette, best_result_by_calinski_harabasz, best_result_by_davies_bouldin

### **1. Знакомство со Структурой Данных**

Первым делом необходимо понять, с какими данными предстоит работать, и произвести базовую предобработку данных — перевести признаки в необходимые для дальнейшей работы форматы.

Познакомьтесь с исходными данными поближе:

* Проведите статистический анализ исходных данных, посмотрев на основные диапазоны исходных признаков.
* Узнайте, сколько уникальных клиентов совершали транзакции в указанный период.
* Узнайте, из каких стран совершались транзакции.
* Исследуйте данные на наличие пропусков и дубликатов.
* Переведите столбцы в корректные форматы (например, даты в формат datetime).

In [ ]:
data = pd.read_csv('./data/data.csv',
                   encoding='ISO-8859-1',
                   dtype={'CustomerID': str,
                          'InvoiceID': str}
)

f'Dimensions: {data.shape}'

In [ ]:
data.head(2)

In [ ]:
#convert to datetime
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

print(f"Date Interval: from {data['InvoiceDate'].dt.date.min()} to {data['InvoiceDate'].dt.date.max()}")

### **2. Преобразование, Очистка и Анализ Данных**

#### **2.1. Преобразование и Очистка данных о Транзакциях**

##### 2.1.1 Пропуски

Пропуски в столбце с идентификатором клиента (CustomerID) и описанием товара свидетельствуют о некорректных/незавершённых транзакциях. Удалите их из данных.

**Примечание.** Если посмотреть на распределение пропусков в столбцах Description и CustomerID, то можно заметить, что достаточно удалить строки, содержащие пропуски в столбце CustomerID, тогда пропуски в столбце Description удаляются автоматически.


In [ ]:
#check that there are no null values in the data set
data.isnull().sum()[data.isnull().sum() > 0]

In [6]:
#delete all records with any blanks
data = data.dropna(axis=0,
                   how='any')

In [ ]:
#check that there are no null values in the data set
data.isnull().sum()[data.isnull().sum() > 0]

In [ ]:
f'New Dimensions: {data.shape}'

##### 2.1.2. Дубликаты

Проверьте данные на наличие дубликатов. Удалите их из данных.


In [ ]:
dupl_columns = list(data.columns)

d_mask = data.duplicated(subset=dupl_columns)
d_duplicates = data[d_mask]
print(f'Number of Duplicates: {d_duplicates.shape[0]}')

In [ ]:
#create a new table free of duplicates
data = data.drop_duplicates(subset=dupl_columns)
print(f'New Dimensions without Duplicates: {data.shape}')

##### 2.1.3. Транзакции с отрицательным количеством товара

Из приведённых выше инсайтов о данных мы узнали все особенности отмены заказов. Когда мы будем сегментировать клиентов, мы будем определять их покупательскую способность. При расчёте покупательской способности нам будет очень важно учесть возвраты этого клиента.

Чтобы подсчитать количество возвратов, для начала нам надо определить, сколько уникальных товаров указано в транзакции (корзине) для каждой уникальной пары «клиент — заказ»:


In [ ]:
temp = data.groupby(by=['CustomerID', 'InvoiceNo'], as_index=False)['InvoiceDate'].count()

nb_products_per_basket = temp.rename(columns = {'InvoiceDate': 'Number of Products'})
nb_products_per_basket.head()

**Примечание.** Более 16 % уникальных заказов являются возвратами. Интересный факт: если мы подсчитали количество транзакций, содержащих признак возврата, в изначальной таблице, где на каждый уникальный товар заведена отдельная строка, то мы получили бы, что количество возвратов менее 1 %. Однако это число было бы некорректным.

Создайте в ваших данных о транзакциях признак `QuantityCanceled`. Этот признак будет указывать на количество отменённого впоследствии товара для каждой транзакции, на которую найдётся противоположная ей транзакция с возвратом. Для транзакций, для которых не было возвратов, признак будет равен 0. Для транзакций, для которых указано отрицательное количество товара, но в данных отсутствует контрагент, признак будет пустым.

В качестве вспомогательного инструмента мы подготовили для вас функцию `get_quantity_canceled()`. Эта функция принимает на вход таблицу с транзакциями и возвращает объект `Series` — столбец, в котором указано количество отменённого впоследствии товара для каждой транзакции. Если транзакция не имеет контрагента, этот признак помечается как `NaN`.

Отметим, что эта функция не учитывает сложный случай, когда количество отменённого товара в транзакции-возврате больше, чем количество товара, которое указано в любой из отдельных транзакций на покупку (это случай, когда клиент сделал несколько заказов одного и того же товара, а потом оформил возврат на все товары разом). При желании вы можете самостоятельно модифицировать функцию для предобработки, чтобы учесть этот пограничный случай.

In [12]:
data['QuantityCanceled'] = get_quantity_cancelled(data)

Узнайте, сколько транзакций в данных не имеют контрагентов, и, если их количество невелико, удалите их из данных.

Когда вы разобрались с возвратами, удалите транзакции с отрицательным количеством товара — они нам больше не нужны.

In [ ]:
print(f'{data.QuantityCanceled.isnull().sum()} transactions have no counterparty')

In [14]:
#delete all transactions without a counterparty
data = data.dropna(axis=0,
                   how='any')

In [15]:
list_index = []
for index, col in data.iterrows():
    if col['Quantity'] - col['QuantityCanceled'] == 0:
        list_index.append(index)
    if col['Quantity'] < 0:
        list_index.append(index)

data = data.drop(index=list_index,
                 axis=0)

In [ ]:
print(f'New Dimensions: {data.shape}')

##### 2.1.4. Специализированные транзакции

Следующая задача — обработать специальные виды транзакций, которые обозначаются латинскими буквами.

В данных присутствует шесть специальных видов транзакций. С помощью регулярных выражений найдите такие коды товаров (StockCode), которые начинаются с латинских букв (при этом коды могут содержать цифры).

**Подсказка.** В качестве шаблона для поиска используйте строку '^[a-zA-Z]+'.

Чтобы понять, что означают эти коды, можно заглянуть в столбец с описанием (Description), например POST означает почтовые расходы, C2 — расходы на транспортировку, BANK CHARGES — банковские расходы.

Специальные операции не характеризуют покупательскую способность клиентов, так как не относятся напрямую к их покупкам, поэтому такие записи нам не нужны. Удалите все специальные транзакции из таблицы.

In [ ]:
#search for special transactions
sp_char = '^[a-zA-Z]+'

special_trans = data.loc[data['StockCode'].str.contains(sp_char,
                                                        regex=True)]

special_trans['StockCode'].unique()

In [18]:
#find the index of special transactions
sp_idx = data.loc[data['StockCode'].str.contains(sp_char,
                                                 regex=True)].index

data = data.drop(index=sp_idx,
                 axis=0)

In [ ]:
print(f'New Dimensions: {data.shape}')

##### 2.1.5. Транзакции с товарами без стоимости

При просмотре описательных статистик можно заметить, что на некоторые товары установлена цена в 0 фунтов стерлингов. Таких транзакций оказывается менее 1 % — можно удалить их.

In [20]:
zero_price = data[data['UnitPrice'] == 0].index

data = data.drop(index=zero_price,
                 axis=0)

##### 2.1.6. Общая стоимость товаров в транзакции

Добавьте в ваш датасет общую цену заказа (TotalPrice). Она рассчитывается как:
 
 **общая цена = цена за единицу товара * (количество товаров в заказе - количество возвращённых товаров).**

Этот признак впоследствии поможет вам рассчитать покупательскую способность каждого из клиентов.

In [21]:
data['TotalPrice'] = pd.Series(np.zeros(data.shape[0]),
                               index=data.index)

for index, col in data.iterrows():
    data['TotalPrice'].loc[index] = col['UnitPrice'] * (col['Quantity'] - col['QuantityCanceled'])

In [22]:
#export the table
data.to_csv('data/transformed.csv',
            index=False)

In [23]:
new_data = pd.read_csv('data/transformed.csv')

In [ ]:
new_data.head(2)

In [ ]:
print(f'Dimensions: {new_data.shape}')

Помимо рекомендованных преобразований, вы также можете придумать собственные и применить их к исходным данным.
После завершения предобработки сохраните результат очищения данных в отдельный файл, чтобы впоследствии вам не приходилось повторять эти действия.


#### **2.2. Разведывательный Анализ**

После предобработки исходных данных произведите разведывательный анализ и исследуйте транзакции, ответив на следующие вопросы:

* Клиенты из каких стран покупают больше и чаще?
* Какие страны приносят наибольшую сезонную выручку?
* Присутствует ли в продажах сезонность (когда покупают чаще)?
* Сгруппируйте данные по датам и часам совершения транзакции и найдите количество заказов на каждый день-час. Затем найдите среднее количество ежедневно поступающих заказов в каждый из часов.
* Каково распределение среднего количества ежедневно поступающих заказов по времени суток (часу совершения транзакции)?

**Примечание.** Вы можете сформулировать и другие вопросы. Главная цель — извлечь максимум понятной информации из исходных данных.

Свои рассуждения сопроводите графиками и диаграммами.



Клиенты из каких стран покупают больше и чаще?

In [ ]:
#top 5 countries with top revenue
df_total_revenue = pd.DataFrame(new_data.groupby('Country')['TotalPrice'].sum().sort_values(ascending=False)[:5])
df_total_revenue

In [27]:
#create two charts side by side
fig = make_subplots(
    rows=2,
    cols=1,
    subplot_titles=('Distribution of Total Revenue by Top 5 Countries', 
                    'Distribution of Total Revenue by Top 4 Countries')
    )


#create bar chart
fig.add_trace(go.Bar(
    x=df_total_revenue.index,
    y=df_total_revenue['TotalPrice']),
              row=1,
              col=1)

#create bar chart
fig.add_trace(go.Bar(
    x=df_total_revenue[1:].index,
    y=df_total_revenue[1:]['TotalPrice']),
              row=2,
              col=1)

#set characteristics
fig.update_layout(width=1100,
                  height=500,
                  showlegend=False)

#fig.show()

#export the graph
fig.write_html('plotly_graphs/2.2.1a_bar_total_revenue_by_country.html')

**Вывод**: \
Наибольший доход поступает из Великобритании, доход от 2 до 4 других стран-лидеров превышает всего лишь более 200 тыс., в то время как доход Великобритании составляет почти 6,8 млн.

In [28]:
#distribution of revenue by country
uk_rev = new_data[new_data['Country'] == 'United Kingdom']
netherlands_rev = new_data[new_data['Country'] == 'Netherlands']
eire_rev = new_data[new_data['Country'] == 'EIRE']
germany_rev = new_data[new_data['Country'] == 'Germany']
france_rev = new_data[new_data['Country'] == 'France']

#create side-by-side plots
fig = make_subplots(rows=5,
                    cols=1)

#first graph
fig.add_trace(
    go.Box(x=uk_rev['TotalPrice'], name='United Kingdom'),
    row=1, col=1
)

#second graph
fig.add_trace(
    go.Box(x=netherlands_rev['TotalPrice'], name='Netherlands'),
    row=2, col=1
)

#third graph
fig.add_trace(
    go.Box(x=eire_rev['TotalPrice'], name='EIRE'),
    row=3, col=1
)

#fourth graph
fig.add_trace(
    go.Box(x=germany_rev['TotalPrice'], name='Germany'),
    row=4, col=1
)

#fifth graph
fig.add_trace(
    go.Box(x=france_rev['TotalPrice'], name='France'),
    row=5, col=1
)

#set the same axis limits
fig.update_xaxes(range=[0, 200])


#set characteristics
fig.update_layout(height=500,
                  width=1100,
                  showlegend=False,
                  title_text='Distribution of Revenue by Country',
                  
)

#fig.show()

#export the graph
fig.write_html('plotly_graphs/2.2.1b_box_revenue_by_country.html')

**Вывод**: \
Средний доход от одной покупки не превышает отметки 20,0 для трех ведущих стран, а для Великобритании он даже ниже — 10,2.
Нидерланды — страна с самым высоким средним доходом и самым большим диапазоном между самой низкой и самой высокой ценой покупки. \
Во всех ведущих странах наблюдается большое количество выбросов, указывающих на разное покупательское поведение.

In [ ]:
#top 5 countries with top sales
df_total_sales = pd.DataFrame(new_data.groupby('Country').count().sort_values(by='InvoiceNo',
                                                                              ascending=False)[:5]).iloc[:, :1]
df_total_sales

In [30]:
#create two charts side by side
fig = make_subplots(
    rows=2,
    cols=1,
    subplot_titles=('Distribution of Total Sales by Top 5 Countries', 
                    'Distribution of Total Sales by Top 4 Countries')
    )

#create bar chart
fig.add_trace(go.Bar(
    x=df_total_sales.index,
    y=df_total_sales['InvoiceNo']),
              row=1,
              col=1)

#create bar chart
fig.add_trace(go.Bar(
    x=df_total_sales[1:].index,
    y=df_total_sales[1:]['InvoiceNo']),
              row=2,
              col=1)

#set characteristics
fig.update_layout(width=1100,
                  height=500,
                  showlegend=False)

#fig.show()

#export the graph
fig.write_html('plotly_graphs/2.2.2_bar_total_sales_by_country.html')

**Вывод**: \
Наибольшее количество покупок поступает из Великобритании, доход от 2 до 4 других стран-лидеров превышает всего лишь более 7 тыс., в то время как количество покупок Великобритании составляет более 345 тыс.

Какие страны приносят наибольшую сезонную выручку?

In [31]:
#convert to datetime
new_data['InvoiceDate'] = pd.to_datetime(new_data['InvoiceDate'])

#create the quarter
new_data['Quarter'] = new_data['InvoiceDate'].dt.quarter

#get the data for the 1st quarter
df_quarter_one = new_data[new_data['Quarter'] == 1]
df_quarter_one_rvn = df_quarter_one.groupby(by=['Country', 'Quarter'])['TotalPrice'].sum().sort_values(ascending=False)[:5]
df_quarter_one_rvn = df_quarter_one_rvn.unstack().reset_index().rename_axis(None, axis=1)
df_quarter_one_rvn = df_quarter_one_rvn.rename(columns={1: 'TotalPrice'})

#get the data for the 2nd quarter
df_quarter_two = new_data[new_data['Quarter'] == 2]
df_quarter_two_rvn = df_quarter_two.groupby(by=['Country', 'Quarter'])['TotalPrice'].sum().sort_values(ascending=False)[:5]
df_quarter_two_rvn = df_quarter_two_rvn.unstack().reset_index().rename_axis(None, axis=1)
df_quarter_two_rvn = df_quarter_two_rvn.rename(columns={2: 'TotalPrice'})

#get the data for the 3rd quarter
df_quarter_three = new_data[new_data['Quarter'] == 3]
df_quarter_three_rvn = df_quarter_three.groupby(by=['Country', 'Quarter'])['TotalPrice'].sum().sort_values(ascending=False)[:5]
df_quarter_three_rvn = df_quarter_three_rvn.unstack().reset_index().rename_axis(None, axis=1)
df_quarter_three_rvn = df_quarter_three_rvn.rename(columns={3: 'TotalPrice'})

#get the data for the 4th quarter
df_quarter_four = new_data[new_data['Quarter'] == 4]
df_quarter_four_rvn = df_quarter_four.groupby(by=['Country', 'Quarter'])['TotalPrice'].sum().sort_values(ascending=False)[:5]
df_quarter_four_rvn = df_quarter_four_rvn.unstack().reset_index().rename_axis(None, axis=1)
df_quarter_four_rvn = df_quarter_four_rvn.rename(columns={4: 'TotalPrice'})

In [32]:
#create two charts side by side
fig = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=('Distribution of Quarter 1 Sales by Top 5 Countries', 
                    'Distribution of Quarter 2 Sales by Top 5 Countries',
                    'Distribution of Quarter 3 Sales by Top 5 Countries',
                    'Distribution of Quarter 4 Sales by Top 5 Countries'
                    )
    )


#1st graph
fig.add_trace(
    go.Bar(x=df_quarter_one_rvn['TotalPrice'],
           y=df_quarter_one_rvn['Country'],
           orientation='h'),
    row=1, col=1
)

#2nd graph
fig.add_trace(
    go.Bar(x=df_quarter_two_rvn['TotalPrice'],
           y=df_quarter_two_rvn['Country'],
           orientation='h'),
    row=1, col=2
)

#3rd graph
fig.add_trace(
    go.Bar(x=df_quarter_three_rvn['TotalPrice'],
           y=df_quarter_three_rvn['Country'],
           orientation='h'),
    row=2, col=1
)

#4th graph
fig.add_trace(
    go.Bar(x=df_quarter_four_rvn['TotalPrice'],
           y=df_quarter_four_rvn['Country'],
           orientation='h'),
    row=2, col=2
)

#set the same axis limits
fig.update_xaxes(range=[0, 2550000])

#set characteristics
fig.update_layout(width=1180,
                  height=500,
                  showlegend=False)

#fig.show()

#export the graph
fig.write_html('plotly_graphs/2.2.3_bar_total_sales_by_country_by_quarter.html')

Присутствует ли в продажах сезонность (когда покупают чаще)?

**Вывод**: \
Клиенты из Великобритании демонстрируют кумулятивную модель поведения: каждый квартал имеет более высокие продажи, чем предыдущий; другие страны демонстрируют иные модели поведения, например, у EIRE самые высокие покупки в 3-м квартале. \
Не все страны остаются в первой пятерке в течение всего года, если рассматривать квартал за кварталом.

Сгруппируйте данные по датам и часам совершения транзакции и найдите количество заказов на каждый день-час. Затем найдите среднее количество ежедневно поступающих заказов в каждый из часов.

In [33]:
#create new datetime features: month, day of the month, week day, and hour
new_data['Month'] = new_data['InvoiceDate'].dt.month
new_data['Day'] = new_data['InvoiceDate'].dt.day
new_data['WeekDay'] = new_data['InvoiceDate'].dt.dayofweek
new_data['Hour'] = new_data['InvoiceDate'].dt.hour

In [34]:
#count the sales by month
df_monthly_sales = new_data.groupby('Month')['InvoiceNo'].count().round()

#create a bar chart
fig = px.bar(
    data_frame=df_monthly_sales,
    x=df_monthly_sales.index,
    y='InvoiceNo',
    color='InvoiceNo',
    width=1100,
    height=500,
    color_continuous_scale='brwnyl',
    title='Distribution of Monthly Sales',
)

#set the title of the axis and y-axis limits
fig.update_layout(
    xaxis_title_text='Month',
    yaxis_title_text='Number of Sales',
)

#fig.show()

#export the graph
fig.write_html('plotly_graphs/2.2.4a_bar_sales_by_month.html')

**Вывод**: \
Самые высокие продажи приходятся на 4-й квартал, а самые высокие — на ноябрь, т.е. вероятнее всего, клиенты покупают заранее к Рождеству и Новому Году.

In [35]:
#count the sales by day of the month
df_daily_sales = new_data.groupby('Day')['InvoiceNo'].count().round()

#create a bar chart
fig = px.bar(
    data_frame=df_daily_sales,
    x=df_daily_sales.index,
    y='InvoiceNo',
    color='InvoiceNo',
    width=1100,
    height=500,
    color_discrete_sequence='olive',
    title='Distribution of Daily Sales',
)

#set the title of the axis and y-axis limits
fig.update_layout(
    xaxis_title_text='Day',
    yaxis_title_text='Number of Sales',
)

#fig.show()

#export the graph
fig.write_html('plotly_graphs/2.2.4b_bar_sales_by_day.html')

**Вывод**: \
Довольно случайное распределение продаж по дням, трудно сделать суждение.

In [36]:
#count the sales by day of the week
df_week_sales = new_data.groupby('WeekDay')['InvoiceNo'].count().round()

#create a bar chart
fig = px.bar(
    data_frame=df_week_sales,
    x=df_week_sales.index,
    y='InvoiceNo',
    color='InvoiceNo',
    width=1100,
    height=500,
    color_discrete_sequence='salmon',
    title='Distribution of Week Day Sales',
)

#set the title of the axis and y-axis limits
fig.update_layout(
    xaxis_title_text='Week Day',
    yaxis_title_text='Number of Sales',
)

#fig.show()

#export the graph
fig.write_html('plotly_graphs/2.2.4c_bar_sales_by_weekday.html')

**Вывод**: \
Большинство покупок совершается в начале недели с понедельника по четверг, растет кумулятивно, с самым высоким показателем в четверг.
По данным в субботу не было совершено ни одной покупки.

In [37]:
# строим сводную таблицу и вычисляем по ней среднее количество заказов в час
hourly_sales = new_data.pivot_table(
    values='StockCode',
    index='InvoiceDate',
    columns='Hour',
    aggfunc='count', 
    fill_value=0
).mean().round(2)

#create a bar chart
fig = px.bar(
    data_frame=hourly_sales,
    x=hourly_sales.index,
    y=hourly_sales.values,
    color=hourly_sales.values,
    width=1100,
    height=500,
    color_discrete_sequence='magenda',
    title='Distribution of Average Hourly Sales',
)

#set the title of the axis and y-axis limits
fig.update_layout(
    xaxis_title_text='Hour',
    yaxis_title_text='Number of Average Sales',
)

#fig.show()

#export the graph
fig.write_html('plotly_graphs/2.2.4d_bar_average_sales_by_hour.html')

Каково распределение среднего количества ежедневно поступающих заказов по времени суток (часу совершения транзакции)? 

**Вывод:** \
Наибольшее количество заказов совершается в период с 12.00 до 13.00, т.е., скорее всего, из-за того, что клиенты совершают заказы в обеденный перерыв и имеют свободное время в выходные дни, и то, что среднее количество по часам имеет форму нормального распределения.

#### **2.3. Построение RFM-таблицы и поиск RFM-выбросов**

Мы добрались до самой интересной части нашей задачи. Нам предстоит сформировать признаки, на основе которых мы будем производить сегментацию клиентов.

Для этого давайте познакомимся с очень популярным методом для анализа потребительской ценности под названием RFM. 

<center> <img src=https://miro.medium.com/max/1400/1*uYQjy9SUjW7iWHc2gGanQQ.png align="right" width="400"/> </center>

Метод заключается в группировке клиентов на основе следующих параметров:
* Recency (Давность) — давность последней покупки клиента;
* Frequency (Частота) — общее количество покупок клиента;
* Monetary Value (Денежная ценность) — сколько денег потратил клиент.

Суть RFM-анализа состоит в том, что мы разделяем всех клиентов на группы в зависимости от того, как давно они сделали последнюю покупку, как часто покупали и насколько большой была сумма их заказов.

Например, вот так может выглядеть интерпретация кластеров для случая RF-сегментации (анализа на основе давности и частоты заказов клиента):

<img src=https://retailrocket.ru/wp-content/uploads/2017/06/rfm-1.png>

Задача маркетологов — вести клиента в зону лояльных.

Мы можем рассчитать RFM-характеристики для каждого из клиентов в нашем датасете и на их основе с помощью методов кластеризации построить подобные сегменты клиентов, привязанные к нашим данным.

Чтобы получить RFM-таблицу, нам необходимо сгруппировать данные по идентификаторам клиента и рассчитать следующие  агрегированные характеристики:

* Recency для i-го клиента рассчитывается как разница между датой и временем последнего заказа и точкой отсчёта, переведённая в дни:
    $$t_0-max(t_{i1}, t_{i2},..., t_{iM})$$

    где $t_{ij}$ — дата и время совершения i-ым клиентом своей j-ой покупки.

    В качестве точки отсчёта $t_0$ берём дату на один день «старше», чем все наши данные. Это будет 10 декабря 2011 года (в формате datetime — '2011-12-10 00:00:00').

* Frequency рассчитывается как общее количество уникальных заказов, которые совершил i-ый клиент.
* Monetary Value рассчитывается как общая сумма денег, которую i-ый клиент потратил на наши товары (с учётом возвратов).

Когда вы рассчитаете все характеристики, не забудьте дать столбцам результирующей таблицы соответствующие названия.

In [38]:
recency_data = new_data.groupby('CustomerID')['InvoiceDate'].max().reset_index()

recency_data['Recency'] = pd.to_datetime('2011-12-10') - recency_data['InvoiceDate']

recency_data['Recency'] = recency_data['Recency'].dt.days

rfm_table = pd.DataFrame({
    'Recency': recency_data['Recency'].values,
    'Frequency': new_data.groupby('CustomerID')['InvoiceNo'].nunique(),
    'Monetary': new_data.groupby('CustomerID')['TotalPrice'].sum()
})

In [ ]:
print(f"customers who placed an order more than 200 days ago: {rfm_table[rfm_table['Recency'] > 200].shape[0]}")
print(f"average number of orders per year: {round(rfm_table['Frequency'].mean())}")
print(f"purchase amount by customer №12360: £{round(rfm_table.loc[12360]['Monetary'])}")

После того как вы подготовите RFM-таблицу, визуализируйте каждую из трёх компонент, например, в виде коробчатых диаграмм (boxplot). Вы получите примерно следующие результаты (графики могут отличаться в зависимости от того, как вы выполните предобработку данных):

In [40]:
#create side-by-side plots
fig = make_subplots(rows=1,
                    cols=3)

#first graph: recency
fig.add_trace(
    go.Box(x=rfm_table.iloc[:, 0].values,
           name='Recency'),
    row=1, col=1
)

#second graph: frequency
fig.add_trace(
    go.Box(x=rfm_table.iloc[:, 1].values,
           name='Frequency'),
    row=1, col=2
)

#third graph: monetary
fig.add_trace(
    go.Box(x=rfm_table.iloc[:, 2].values,
           name='Monetary'),
    row=1, col=3
)


#set characteristics
fig.update_layout(height=350,
                  width=1170,
                  showlegend=False,
                  title_text='Distribution of Recency, Frequency, and Monetary',
                  
)

#fig.show()

#export the graph
fig.write_html('plotly_graphs/2.3.1_box_rfm_features.html')

Что интересного здесь можно увидеть? Есть клиенты с аномально большим количеством сделанных заказов (более 100 штук), а также клиенты, общая стоимость заказов которых превышает 190 тысяч фунтов стерлингов.

Чем это плохо? Выбросы могут отрицательно сказаться на результатах работы методов кластеризации, неустойчивых к ним, например алгоритма KMeans, поэтому хотелось бы от них избавиться. Однако терять много ценных данных о клиентах тоже не хочется, поэтому ограничимся верхней границей соответствующей квантили уровня 0.95. Таким образом, мы удалим данные тех клиентов, для которых значение параметра Frequency или параметра Monetary выше, чем у 95 % клиентов.


In [ ]:
#set the limits for frequency and monetary
freq_bound = np.quantile(rfm_table['Frequency'], 0.95)
monetary_bound = np.quantile(rfm_table['Monetary'], 0.95)

#set the limits
rfm_table = rfm_table[~((rfm_table['Frequency'] > freq_bound) | (rfm_table['Monetary'] > monetary_bound))]
rfm_table.shape

In [42]:
#create side-by-side plots
fig = make_subplots(rows=1,
                    cols=3)

#first graph: recency
fig.add_trace(
    go.Box(x=rfm_table.iloc[:, 0].values,
           name='Recency'),
    row=1, col=1
)

#second graph: frequency
fig.add_trace(
    go.Box(x=rfm_table.iloc[:, 1].values,
           name='Frequency'),
    row=1, col=2
)

#third graph: monetary
fig.add_trace(
    go.Box(x=rfm_table.iloc[:, 2].values,
           name='Monetary'),
    row=1, col=3
)


#set characteristics
fig.update_layout(height=350,
                  width=1170,
                  showlegend=False,
                  title_text='Distribution of Recency, Frequency, and Monetary',
                  
)

#fig.show()

#export the graph
fig.write_html('plotly_graphs/2.3.2_box_rfm_features.html')

**Вывод:** \
После удаления выбросов графики имеют характерную форму, но выбросы все еще присутствуют.

In [43]:
#create 3D scatter plot
fig = px.scatter_3d(rfm_table,
                    x='Recency',
                    y='Frequency',
                    z='Monetary',
                    color='Monetary',
                    size='Monetary'
                    )

#set characteristics
fig.update_layout(height=750,
                  width=900,
                  showlegend=False,
                  title_text='Recency, Frequency, and Monetary',                  
)

#fig.show()

#export the graph
fig.write_html('plotly_graphs/3.1_3d_scatter_rfm_features.html')

**Вывод:** \
По массе точек сложно сказать, сколько кластеров необходимо — скорее даже кажется, нужно проанализировать данные через несколько моделей кластеризации для принятия решения.

### **3. Моделирование и Оценка Качества Моделей**
#### Обучение без Учителя

#### **3.1. Кластеризация на основе RFM-Характеристик**

Чтобы результаты кластеризации было удобнее интерпретировать, вы можете уменьшить размерность исходных признаков до двух компонент.

**Подсказка.** Чтобы методы понижения размерности работали стабильно, данные необходимо стандартизировать/нормализовать. Для удобства оберните эти шаги по предобработке данных в pipeline.

Произведите предобработку исходных данных. На основе RFM-признаков кластеризуйте клиентов онлайн-магазина подарков с помощью известных вам методов (используйте минимум три метода).

Подберите оптимальное количество кластеров для выбранных методов с помощью коэффициента силуэта, перебирая возможные значения от 3 до 10 включительно (большее или меньшее количество кластеров будет нерелеватно для маркетинга). 

Выберите алгоритм с наибольшим коэффициентом силуэта, сделайте предсказание меток кластеров.


In [44]:
#initialize the scaler
s_scaler = preprocessing.StandardScaler()

#standardize the data
rfm_table_scaled = s_scaler.fit_transform(rfm_table)

##### 3.1.1.0. Principal Component Analysis (PCA)

нужно использовать снижение размерности, чтобы упростить большой набор данных до меньшего, при этом сохранить значимые закономерности и тенденции

In [ ]:
#initialize the object class
pca = decomposition.PCA()

#model learning on standardised data set
pca.fit_transform(rfm_table_scaled)

#explained variance ratio on the number of features
pca.explained_variance_ratio_

In [ ]:
#show the % of data covered by the combination of features
plt.plot(range(1, 4),
         pca.explained_variance_ratio_.cumsum(),
         marker='o',
         linestyle='--')
plt.title('Explained Variance by Components')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance');

К сожалению нельзя выбрать 1,5 компонента, чтобы покрыть 80% данных для стандартного решения, поэтому нужно использовать количество компонентов = 2. \
2 компонента описывает >90% данных.

In [47]:
from sklearn import pipeline

#convert the dimensionality reduction into a pipeline
pipeline = pipeline.Pipeline([
    ('scaler', preprocessing.StandardScaler()), 
    ('pca', decomposition.PCA(n_components=2))
])

pca_model = pipeline.fit_transform(rfm_table)

In [ ]:
#visualize the space of principal components after decomposition
pca_processed = pd.DataFrame(pca_model,
                             columns=['axis-1',
                                      'axis-2'])

#create the scatterplot
sns.scatterplot(data=pca_processed,
                x='axis-1',
                y='axis-2');

##### 3.1.1.1. K-Means

*Внутренние Меры*

**Silhouette Score** \
compare the change in silhouette score for standardised data set with all features and dimensionally reduced data set with 2 features

In [ ]:
#show the change in silhouette score
clusters_by_silhouette('k-means', rfm_table_scaled, 3, 11)

In [ ]:
#show the change in silhouette score
clusters_by_silhouette('k-means', pca_processed, 3, 11)

In [ ]:
print(f'All Features, Best Result: {best_result_by_silhouette('k-means', rfm_table_scaled, 3, 11)}')
print(f'Reduced Features, Best Result: {best_result_by_silhouette('k-means', pca_processed, 3, 11)}')

**Вывод:** \
Чем выше оценка, тем лучше, таким образом, оценка предполагает наилучшее количество кластеров = 3. \
Набор данных с уменьшенной размерностью показывает лучшие результаты для модели K-Means: 0.523 > 0.474.

**Calinski-Harabasz Index** \
compare the change in calinski-harabasz score for standardised data set with all features and dimensionally reduced data set with 2 features

In [ ]:
clusters_by_calinski_harabasz('k-means', rfm_table_scaled, 3, 11)

In [ ]:
clusters_by_calinski_harabasz('k-means', pca_processed, 3, 11)

In [ ]:
print(f'All Features, Best Result: {best_result_by_calinski_harabasz('k-means', rfm_table_scaled, 3, 11)}')
print(f'Reduced Features, Best Result: {best_result_by_calinski_harabasz('k-means', pca_processed, 3, 11)}')

**Вывод:** \
Чем выше оценка, тем лучше, таким образом, оценка предполагает наилучшее количество кластеров = 6. \
Набор данных с уменьшенной размерностью показывает лучшие результаты для модели K-Means: 6984.539 > 4467.882.

**Davies-Bouldin Index** \
compare the change in davies-bouldin score for standardised data set with all features and dimensionally reduced data set with 2 features

In [ ]:
clusters_by_davies_bouldin('k-means', rfm_table_scaled, 3, 11)

In [ ]:
clusters_by_davies_bouldin('k-means', pca_processed, 3, 11)

In [ ]:
print(f'All Features, Best Result: {best_result_by_davies_bouldin('k-means', rfm_table_scaled, 3, 11)}')
print(f'Reduced Features, Best Result: {best_result_by_davies_bouldin('k-means', pca_processed, 3, 11)}')

**Вывод:** \
Чем ниже оценка, тем лучше, таким образом, оценка предполагает наилучшее количество кластеров = 3. \
Набор данных с уменьшенной размерностью показывает лучшие результаты для модели K-Means: 0.632 < 0.742.

**K-Means with Clusters suggested by Internal Measures**

In [ ]:
#create a new dataframe to store model results
compare_models_table = rfm_table.copy()

k_means_final_model = cluster.KMeans(n_clusters=3,
                                     init='k-means++',
                                     random_state=42)

#model learning
k_means_final_model.fit(pca_processed)

#list of clusters
km_labels = k_means_final_model.labels_

compare_models_table['KM-Labels'] = km_labels
compare_models_table['KM-Labels'].value_counts()

##### 3.1.1.2. EM-Algorithm

*Внутренние Меры*

**Silhouette Score** \
compare the change in silhouette score for standardised data set with all features and dimensionally reduced data set with 2 features

In [ ]:
clusters_by_silhouette('em-algorithm', rfm_table_scaled, 3, 11)

In [ ]:
clusters_by_silhouette('em-algorithm', pca_processed, 3, 11)

In [ ]:
print(f'All Features, Best Result: {best_result_by_silhouette('em-algorithm', rfm_table_scaled, 3, 11)}')
print(f'Reduced Features, Best Result: {best_result_by_silhouette('em-algorithm', pca_processed, 3, 11)}')

**Вывод:** \
Чем выше оценка, тем лучше, таким образом, оценка предполагает наилучшее количество кластеров = 3. \
Набор данных с уменьшенной размерностью показывает лучшие результаты для модели EM-Algorithm: 0.432 > 0.268.

**Calinski-Harabasz Index** \
compare the change in calinski-harabasz score for standardised data set with all features and dimensionally reduced data set with 2 features

In [ ]:
clusters_by_calinski_harabasz('em-algorithm', rfm_table_scaled, 3, 11)

In [ ]:
clusters_by_calinski_harabasz('em-algorithm', pca_processed, 3, 11)

In [ ]:
print(f'All Features, Best Result: {best_result_by_calinski_harabasz('em-algorithm', rfm_table_scaled, 3, 11)}')
print(f'Reduced Features, Best Result: {best_result_by_calinski_harabasz('em-algorithm', pca_processed, 3, 11)}')

**Вывод:** \
Чем выше оценка, тем лучше, таким образом, оценка предполагает наилучшее количество кластеров = 3. \
Набор данных с уменьшенной размерностью показывает лучшие результаты для модели EM-Algorithm: 4328.155 > 2127.646.

**Davies-Bouldin Index** \
compare the change in davies-bouldin score for standardised data set with all features and dimensionally reduced data set with 2 features

In [ ]:
clusters_by_davies_bouldin('em-algorithm', rfm_table_scaled, 3, 11)

In [ ]:
clusters_by_davies_bouldin('em-algorithm', pca_processed, 3, 11)

In [ ]:
print(f'All Features, Best Result: {best_result_by_davies_bouldin('em-algorithm', rfm_table_scaled, 3, 11)}')
print(f'Reduced Features, Best Result: {best_result_by_davies_bouldin('em-algorithm', pca_processed, 3, 11)}')

**Вывод:** \
Чем ниже оценка, тем лучше, таким образом, оценка предполагает наилучшее количество кластеров = 3. \
Набор данных с уменьшенной размерностью показывает лучшие результаты для модели EM-Algorithm: 0.723 < 1.224.

**EM-Algorithm with Clusters suggested by Internal Measures**

In [ ]:
em_final_model = mixture.GaussianMixture(n_components=3,
                                         random_state=42)

#model learning > list of clusters
predictions = em_final_model.fit_predict(pca_processed)

compare_models_table['EM-Labels'] = predictions
compare_models_table['EM-Labels'].value_counts()

##### 3.1.1.3. Agglomerate Clustering

*Внутренние Меры*

**Silhouette Score** \
compare the change in silhouette score for standardised data set with all features and dimensionally reduced data set with 2 features

In [ ]:
clusters_by_silhouette('agglomerative_clustering', rfm_table_scaled, 3, 11)

In [ ]:
clusters_by_silhouette('agglomerative_clustering', pca_processed, 3, 11)

In [ ]:
print(f'All Features, Best Result: {best_result_by_silhouette('agglomerative_clustering', rfm_table_scaled, 3, 11)}')
print(f'Reduced Features, Best Result: {best_result_by_silhouette('agglomerative_clustering', pca_processed, 3, 11)}')

**Вывод:** \
Чем выше оценка, тем лучше, таким образом, оценка предполагает наилучшее количество кластеров = 3 с linkage = average. \
Набор данных с уменьшенной размерностью показывает лучшие результаты для модели Agglomerative Clustering, но не значительно: 0.477 > 0.475.

**Calinski-Harabasz Index** \
compare the change in calinski-harabasz score for standardised data set with all features and dimensionally reduced data set with 2 features

In [ ]:
clusters_by_calinski_harabasz('agglomerative_clustering', rfm_table_scaled, 3, 11)

In [ ]:
clusters_by_calinski_harabasz('agglomerative_clustering', pca_processed, 3, 11)

In [ ]:
print(f'All Features, Best Result: {best_result_by_calinski_harabasz('agglomerative_clustering', rfm_table_scaled, 3, 11)}')
print(f'Reduced Features, Best Result: {best_result_by_calinski_harabasz('agglomerative_clustering', pca_processed, 3, 11)}')

**Вывод:** \
Чем выше оценка, тем лучше, таким образом, оценка предполагает наилучшее количество кластеров = 3 с linkage = ward. \
Набор данных с уменьшенной размерностью показывает лучшие результаты для модели Agglomerative Clustering: 5850.243 > 3814.718.

**Davies-Bouldin Index** \
compare the change in davies-bouldin score for standardised data set with all features and dimensionally reduced data set with 2 features

In [ ]:
clusters_by_davies_bouldin('agglomerative_clustering', rfm_table_scaled, 3, 11)

In [ ]:
clusters_by_davies_bouldin('agglomerative_clustering', pca_processed, 3, 11)

In [ ]:
print(f'All Features, Best Result: {best_result_by_davies_bouldin('agglomerative_clustering', rfm_table_scaled, 3, 11)}')
print(f'Reduced Features, Best Result: {best_result_by_davies_bouldin('agglomerative_clustering', pca_processed, 3, 11)}')

**Вывод:** \
Чем ниже оценка, тем лучше, таким образом, оценка предполагает наилучшее количество кластеров = 4 с linkage = single. \
Набор данных со всеми компонентами показывает лучшие результаты для модели Agglomerative Clustering: 0.419 < 0.428.

Индекс Дэвиса-Боулдинга обычно выше для выпуклых кластеров, чем для других концепций кластеров, таких как кластеры на основе плотности, подобные тем, что получены из DBSCAN, поэтому для анализа будем опираться на первых два внутренних мер.

**Agglomerative Clustering with Clusters suggested by Internal Measures**

In [96]:
aggl_model_av = cluster.AgglomerativeClustering(n_clusters=6,
                                                linkage='average')

#model learning > list of clusters
y_pred_av = aggl_model_av.fit_predict(pca_processed)

compare_models_table['AC-Labels_Av'] = y_pred_av

##### 3.1.1.4. Density-Based Spatial Clustering of Applications with Noise (DBSCAN)

вычислить k-nearest neighbors (рассчитать расстояние точки до ее k-го ближайшего соседа) для поиска оптимального значения eps

In [ ]:
#find the appropriate value of eps
from sklearn.neighbors import NearestNeighbors

#initialize the value of k for kNN <> MinPts
MinPts = rfm_table.shape[1] * 2
k = MinPts

#add 1 to k to return distance to itself (1st column as 0)
nbrs = NearestNeighbors(n_neighbors=k+1).fit(rfm_table)

#find the distances
dist, ind = nbrs.kneighbors(rfm_table)

#drop 1st column and sort the distances (ASC)
k_dist = np.sort(dist[:, -1])

fig = px.line(k_dist,
              labels={'index': 'Distance Sorted Points',
                      'value': f'{k}-Distance'
              },
              title='Nearest Neighbors Analysis')

#set characteristics
fig.update_layout(width=700,
                  height=450,
                  showlegend=False)

fig.show()

**Вывод**: \
Перегиб на уровне 0.25-0.30, что указывает на хорошее значение для eps.

In [81]:
#create a new dataframe to store model results
#dbscan_table = rfm_table.copy()

dbscan = cluster.DBSCAN(eps=0.25,
                        min_samples=MinPts).fit(rfm_table_scaled)

dbscan_pca = cluster.DBSCAN(eps=0.25,
                            min_samples=MinPts).fit(pca_processed)

compare_models_table['DB-Labels_S'] = dbscan.labels_
compare_models_table['DB-Labels_DR'] = dbscan_pca.labels_

In [ ]:
print(compare_models_table['DB-Labels_S'].nunique(), compare_models_table['DB-Labels_DR'].nunique())

Шумным образцам присваивается метка -1: \
35 кластеров + шум для данных со всеми признаками, и только 3 кластера + шум для набора данных с уменьшенной размерностью.

In [ ]:
y_pred = dbscan_pca.labels_

cl_number_for_dbscan = compare_models_table['DB-Labels_DR'].nunique()-1
print(f'Clusters: {cl_number_for_dbscan}, Silhouette Score: {round(metrics.silhouette_score(pca_processed, y_pred), 3)}')
print(f'Clusters: {cl_number_for_dbscan}, Calinksi-Harabasz Score: {round(metrics.calinski_harabasz_score(pca_processed, y_pred), 3)}')
print(f'Clusters: {cl_number_for_dbscan}, Davies-Bouldin Score: {round(metrics.davies_bouldin_score(pca_processed, y_pred), 3)}')

In [ ]:
compare_models_table.head(2)

##### 3.1.2.0. t-Distributed Stochastic Neighbour Embedding

IDEAL ANSWER?

In [ ]:
# pipeline_tsne = Pipeline([
#     ('scaler', preprocessing.StandardScaler()), 
#     ('tsne', manifold.TSNE(perplexity=50, random_state=100))
# ])
# tsne = pipeline_tsne.fit_transform(rfm_table)
# print('{:.2f}'.format(pipeline_tsne['tsne'].kl_divergence_))

In [ ]:
# rfm_table_processed = pd.DataFrame(tsne, columns=['axis-1', 'axis-2'])

# fig = plt.figure(figsize=(12, 5))
# sns.scatterplot(data=rfm_table_processed, x='axis-1', y='axis-2');

find the best hyperparameters for t-sne model using optuna

In [98]:
import optuna
from optuna import Trial, study, samplers

def optuna_tsne(trial):
  #set hyperparameters
  n_components = trial.suggest_categorical('n_components', [2])
  perplexity = trial.suggest_categorical('perplexity', [5, 10, 20, 30, 40, 50])
  init = trial.suggest_categorical('init', ['random', 'pca'])
  learning_rate = trial.suggest_categorical('learning_rate', [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600])
  early_exaggeration = trial.suggest_categorical('early_exaggeration', [1, 2, 3, 4, 5, 6, 7, 8])
  
  #use the combinations for model build
  model = manifold.TSNE(n_components=n_components,
                        perplexity=perplexity,
                        learning_rate=learning_rate,
                        init=init,
                        early_exaggeration=early_exaggeration)
  
  tsne_results = model.fit(rfm_table_scaled)

  return round(tsne_results.kl_divergence_, 5)

In [ ]:
%%time

#begin hyperparameters selection
#create review object
study_optuna = optuna.create_study(study_name='t-distributed stochastic neighbor embedding',
                                   direction='minimize')

#search for the best combination
study_optuna.optimize(optuna_tsne,
                      n_trials=20)

In [ ]:
print(f'best value: {study_optuna.best_value:.3f}')
print(f'best hyperparameters for t-sne: {study_optuna.best_params}')

model learning on standardised data

In [ ]:
#find the metrics for test data
model_opt_tsne = manifold.TSNE(**study_optuna.best_params,
                               random_state=42)

#model learning
tsne = model_opt_tsne.fit_transform(rfm_table_scaled)

#make a prediction
model_opt_tsne.kl_divergence_

In [ ]:
#create the dataframe to store the results
rfm_table_processed = pd.DataFrame(tsne,
                                   columns=['axis-1', 'axis-2'])

#create the scatterplot to visualize the clusters
sns.scatterplot(data=rfm_table_processed,
                x='axis-1', y='axis-2');

t-SNE сгруппировал наиболее похожие объекты в подобие кластеров.

k-means with t-sne by internal measures

In [ ]:
best_result_by_silhouette('k-means', tsne, 3, 8)

In [ ]:
best_result_by_calinski_harabasz('k-means', tsne, 3, 8)

In [ ]:
best_result_by_davies_bouldin('k-means', tsne, 3, 8)

em-algorithm with t-sne by internal measures

In [ ]:
best_result_by_silhouette('em-algorithm', tsne, 3, 8)

In [ ]:
best_result_by_calinski_harabasz('em-algorithm', tsne, 3, 8)

In [ ]:
best_result_by_davies_bouldin('em-algorithm', tsne, 3, 8)

agglomerative clustering with t-sne by internal measures

In [ ]:
best_result_by_silhouette('agglomerative_clustering', tsne, 3, 8)

In [ ]:
best_result_by_calinski_harabasz('agglomerative_clustering', tsne, 3, 8)

In [ ]:
best_result_by_davies_bouldin('agglomerative_clustering', tsne, 3, 8)

In [ ]:
tree = cluster.KMeans(n_clusters=7)
tree.fit(tsne)

pd.DataFrame((np.unique(tree.labels_,
                        return_counts=True)))

In [ ]:
sns.scatterplot(data=rfm_table_processed,
                x='axis-1', y='axis-2',
                hue=tree.labels_.astype('str'));

#### **3.2. Интерпретация Результатов Кластеризации**

Перейдём к интерпретации полученных кластеров.

##### 3.2.1. Визуализация Кластеров

Визуализируйте результаты в виде 3D-диаграммы с осями Recency, Frequency и Monetary. Проанализируйте полученную диаграмму и попробуйте понять, какие кластеры у вас получились.

In [ ]:
rfm_table['Cluster'] = tree.labels_
rfm_clusters = rfm_table.groupby('Cluster').mean().round(0)
rfm_clusters

In [122]:
#create 3D scatter plot
fig = px.scatter_3d(rfm_table,
                    x='Recency',
                    y='Frequency',
                    z='Monetary',
                    color='Cluster',
                    size='Cluster'
                    )

#set characteristics
fig.update_layout(height=750,
                  width=900,
                  showlegend=False,
                  title_text='Recency, Frequency, and Monetary',                  
)

#fig.show()

#export the graph
fig.write_html('plotly_graphs/3.2_3d_scatter_rfm_features_with_clusters.html')

##### 3.2.2. Построение Профиля Кластеров

Далее составьте так называемый профиль кластеров. Для этого вам необходимо вернуться от декомпозированных данных (если вы производили понижение размерности) к RFM-таблице (очищенной от выбросов).

Сгруппируйте RFM-таблицу по полученным кластерам и рассчитайте среднее по каждому из признаков.

Чтобы результаты было проще интерпретировать, давайте познакомимся с одним из способов визуализации профиля кластеров — **Radar Chart** (полярная диаграмма, или диаграмма паутины). Это графическое представление значений нескольких эквивалентных категорий в форме паутины.

Radar Chart часто используется при определении профиля кластеров. На концах паутины откладываются оси, соответствующие признакам, описывающим объекты. На каждой из осей для каждого кластера откладываются средние значения соответствующих характеристик. Соединив точки по осям, мы получаем многоугольник. 

Пример полярной диаграммы для задачи кластеризации учеников по интересам:

<img src=https://www.datanovia.com/en/wp-content/uploads/2020/12/radar-chart-in-r-customized-fmstb-radar-chart-1.png width=500>

На этой диаграмме мы видим визуализацию признаков для одного из кластеров. Видно, что ученики, принадлежащие этому кластеру, в большей степени увлекаются музыкой (Music), а в меньшей — программированием (Programm).

В модуле `graph_objects` библиотеки `plotly` есть встроенная функция `Scatterpolar`, которая позволяет построить полярную диаграмму. На основе этой функции мы подготовили для вас функцию `plot_cluster_profile()`, которая позволяет визуализировать профиль каждого из кластеров в виде полярной диаграммы. У неё есть два параметра: `grouped_data` — сгруппированные по кластерам характеристики объектов (клиентов), `n_clusters` — количество кластеров.

Главное условие использования полярной диаграммы — все признаки должны быть приведены к единому масштабу с помощью нормализации, где 1 будет означать максимум, а 0 — минимум. Шаг с нормализацией мы также добавили в функцию `plot_cluster_profile()`.


In [ ]:
plot_cluster_profile(rfm_clusters, 7)

Теперь у вас есть удобный инструмент для анализа профиля кластеров. Воспользуйтесь им, чтобы проинтерпретировать результаты, полученные на предыдущем шаге.

# FINISH OFF

**Вывод**: \
Группа **БЕДНЫЕ НОВИЧКИ** не делают значительного вклада в данные и имеют одинаково малые показатели по всем 3 осям. Группа **БЕДНЫЕ СПЯЩИЕ** делали заказы очень давно, но не часто и не принесли значительной прибыли магазину. А группа **БОГАТЫЕ ЛОЯЛЬНЫЕ** делают заказы часто и тратят много денег. 

### **4. Моделирование и Оценка Качества Моделей**
#### Обучение с Учителем

In [125]:
#select the features
X = rfm_table.drop(columns='Cluster')
y = rfm_table['Cluster']

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y,
                                                                    test_size=0.3,
                                                                    random_state=42)

In [126]:
#set the clusters
cluster_report = ['Cluster 1', 'Cluster 2', 'Cluster 3', 'Cluster 4', 'Cluster 5', 'Cluster 6', 'Cluster 7']

#### **4.1. Random Forest Classifier**

In [127]:
#use the Optuna method to look for best hyper parameters
def optuna_rf(trial):
  #set hyperparameters
  n_estimators = trial.suggest_categorical('n_estimators', [50, 100, 150, 200])
  max_depth = trial.suggest_categorical('max_depth', [3, 5, 7])
  min_samples_leaf = trial.suggest_categorical('min_samples_leaf', [2, 3, 4])
  criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
  max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
  
  #use the combinations for model build
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          criterion=criterion,
                                          max_features=max_features,
                                          random_state=42)
  
  #model learning through cross-validation
  score = model_selection.cross_val_score(
    model,
    X=X_train,
    y=y_train,
    cv=5,
    scoring='f1_micro', 
    n_jobs=-1).mean()

  return score

In [ ]:
%%time
#begin hyperparameters selection
#create review object
study_optuna_rf = optuna.create_study(study_name='random forest classifier',
                                       direction='maximize')

#search for the best combination
study_optuna_rf.optimize(optuna_rf,
                         n_trials=10)

In [ ]:
print(f'best value: {round(study_optuna_rf.best_value, 3)}')
print(f'best hyperparameters for random forest: {study_optuna_rf.best_params}')

In [130]:
#find the metrics for test data
model_opt_rf = ensemble.RandomForestClassifier(**study_optuna_rf.best_params,
                                               random_state=42,
                                               )

#model learning
model_opt_rf.fit(X_train, y_train)

#make a prediction
y_train_pred_rf = model_opt_rf.predict(X_train)
y_test_pred_rf = model_opt_rf.predict(X_test)

In [ ]:
print(metrics.classification_report(y_train,
                                    y_train_pred_rf,
                                    target_names=cluster_report,
                                    digits=5))

print(metrics.classification_report(y_test,
                                    y_test_pred_rf,
                                    target_names=cluster_report,
                                    digits=5))

#### **4.3. Gradient Boosting Classifier**

In [132]:
#use the Optuna method to look for best hyper parameters
def optuna_gb(trial):
  #set hyperparameters
  params = {
    "n_estimators": trial.suggest_categorical('n_estimators', [50, 100, 150, 200]),
    "learning_rate": trial.suggest_categorical('learning_rate', [0.001, 0.01, 0.05, 0.1, 0.15, 0.2]),    
    "max_depth": trial.suggest_categorical('max_depth', [3, 5, 7]),
    "max_features": trial.suggest_categorical('max_features', ['sqrt', 'log2']),
    "min_samples_leaf": trial.suggest_categorical('min_samples_leaf', [2, 3, 4]),
    "loss" : "log_loss",
    "random_state": 42,
    }

  #use the combinations for model build
  model = ensemble.GradientBoostingClassifier(**params)
  
  #model learning through cross-validation
  score = model_selection.cross_val_score(
    model,
    X_train,
    y_train,
    cv=5,
    scoring='f1_micro',
    n_jobs=-1).mean()

  return score

In [ ]:
%%time
#begin hyperparameters selection
#create review object
study_optuna_gb = optuna.create_study(study_name='gradient boosting classifier',
                                   direction='maximize')

#search for the best combination
study_optuna_gb.optimize(optuna_gb,
                         n_trials=10)

In [ ]:
print(f'best value: {round(study_optuna_gb.best_value, 3)}')
print(f'best hyperparameters for random forest: {study_optuna_gb.best_params}')

In [135]:
#find the metrics for test data
model_opt_gb = ensemble.GradientBoostingClassifier(**study_optuna_gb.best_params,
                                                   random_state=42,
                                                   )

#model learning
model_opt_gb.fit(X_train, y_train)

#make a prediction
y_train_pred_gb = model_opt_gb.predict(X_train)
y_test_pred_gb = model_opt_gb.predict(X_test)


In [ ]:
print(metrics.classification_report(y_train,
                                    y_train_pred_gb,
                                    target_names=cluster_report,
                                    digits=5))

print(metrics.classification_report(y_test,
                                    y_test_pred_gb,
                                    target_names=cluster_report,
                                    digits=5))

#### **4.1. Кластеризация на основе RFM-Характеристик**

#### **4.2. Интерпретация Результатов Кластеризации**

### 5. **Выводы и Оформление Работы**

# FINISH OFF